LIBRARY 

In [ ]:
import torch # thư viện học sâu
import torch.nn as nn # thư viện mạng nơ-ron để xây dựng mô hình
import numpy as np # thư viện toán học
import matplotlib.pyplot as plt # thư viện vẽ đồ thị
import torchvision.transforms as transforms # thư viện xử lý ảnh khác với PIL ở chỗ có thể áp dụng trực tiếp trên tensor
from torchvision.utils import make_grid  # hàm để hiển thị ảnh
import json # file json để lưu nhãn
import torchvision.datasets as datasets # thư viện các dataset
import torchvision.models as models # thư viện các mô hình học sâu
import torch.utils.data as data # thư viện xử lý dữ liệu
from torch.utils.data import DataLoader # load dữ liệu lên CPU
from PIL import Image # thư viện xử lý ảnh
import os.path as osp # thư viện os.path để thao tác với đường dẫn file
import random # thư viện random để lấy mẫu ngẫu nhiên
import os # thư viện os để thao tác với file hệ thống

In [2]:
torch.manual_seed(21) 
np.random.seed(21) 
random.seed(21)

PRETRAIN

In [3]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = transforms.Compose([
            transforms.Resize((resize, resize)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    
    def __call__(self, img):
        return self.data_transform(img)

In [10]:
class CustomDataset(data.Dataset):
    def __init__(self, root_dir, transform=None, classes=("good", "defect")):
        self.root_dir = root_dir 
        self.transform = transform
        self.classes = list(classes)
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)} # gán nhãn với chỉ số
        self.samples = []
        for cls in self.classes:
            cls_dir = osp.join(root_dir, cls)
            if not osp.isdir(cls_dir):
                continue
            for filename in os.listdir(cls_dir):
                if filename.endswith('.jpg'):
                    img_path = osp.join(cls_dir, filename) 
                    label = self.class_to_idx[cls]
                    self.samples.append((img_path, label))
        if len(self.samples) == 0:
            raise RuntimeError(f"Lỗi không tìm thấy ảnh trong {root_dir}.")
    def __len__(self):
        return len(self.samples)  
    def __getitem__(self, idx):  
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(label, dtype=torch.long)
        return img, label

In [ ]:
from torch.utils.data import DataLoader
transform = ImageTransform(
    resize=224,
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

dataset = CustomDataset(root_dir= './data/image/', transform=transform)

dataloader = DataLoader(
    dataset,          # dataset cần load
    batch_size=16,    # số lượng ảnh trong 1 batch (tùy GPU/CPU)
    shuffle=True,     # xáo trộn dữ liệu mỗi epoch
    num_workers=2,    # số luồng CPU đọc dữ liệu song song
    pin_memory=True   # tăng tốc độ khi dùng GPU
)
for imgs, labels in dataloader:
    print("Kích thước batch ảnh:", imgs.shape)
    print("Kích thước batch nhãn:", labels.shape)
    break  # chỉ in 1 batch để kiểm tra